In [3]:
import torch
import torchvision.models as models
import torch.nn as nn

Здесь мы проверим свое окружение: доступны ли GPU? Сколько их?

Для абстрагирования от уровня оборудования можно задать устройство, на котором производятся вычисления как `DEVICE` и отправлять на него модель и данные методом `.to(DEVICE)`. В коде это будет выглядеть одинаково и в случае, если устройство - CPU, и в случае графического ускорителя. Удобно и лаконично.

In [4]:
torch.cuda.device_count()

1

In [5]:
DEVICE = torch.device( "cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

Теперь загрузим предобученную модель. Список предварительно обученных моделей, доступных для использования в рамках `torchvision.models` можно посмотреть на странице документации: https://pytorch.org/vision/stable/models.html

Здесь мы загрузим AlexNet:

In [7]:
alexnet = models.alexnet(pretrained=True, progress=False)

Есть много вариантов визуализировать граф нейросети. Мы еще разберем некоторые из этих способов. Однако самый простой - просто вывести текстовое описание на экран:

In [9]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Заметим, что AlexNet состоит из трех частей: сверточной (`features`), операции субдискретизации (`avgpool`) и полносвязного классификатора (`classifier`). Суть подхода Transfer Learning в том, чтобы заменить часть нейросети на новую и обучать её. Например, можно заменить полносвязную часть `classifier` на совершенно новую подсеть, которая, возможно, будет выполнять совершенно новую задачу - например, регрессию. При этом она будет использовать признаки, извлекаемые сверточной подсетью. При этом обычно свёрточную подсеть "замораживают", то есть, запрещают вычислять градиенты функции потерь по весам этой подсети. С нулевыми градиентами эти веса не меняются в процессе градиентной оптимизации вне зависимости от типа оптимизатора (лишь бы он был градиентный).

In [ ]:
# Запрещаем вычислять градиенты функции потерь по всем параметрам AlexNet
for param in alexnet.parameters():
    param.requires_grad = False

# Заменяем полносвязную часть на новую, еще не обученную
alexnet.classifier = torch.nn.Sequential(nn.Linear(in_features=2048, out_features=256, bias=True),
                                         nn.ReLU(),
                                         nn.Linear(in_features=256, out_features=128, bias=True),
                                         nn.ReLU(),
                                         nn.Linear(in_features=128, out_features=3, bias=True))

# Отправляем модель на устройство, на котором будут производиться вычисления
_ = alexnet.to(DEVICE)

Еще один пример - с более сложной моделью. Resnet152 извлекает более богатые признаки, на основании которых его полносвязная часть `fc` выполняет классификацию. У исследователя потенциально решается другая задача - не классификация, не ImageNet. Поэтому имеет смысл заменить полносвязную часть и обучать только ее.

In [10]:
resnet152 = models.resnet152(pretrained=True, progress=False)

In [11]:
resnet152

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Если нужно построить на базе resnet152 какую-то новую модель, которая не строится просто так заменой полносвязной части `fc` на новую, можно заменить `fc` на идентичное преобразование и полученную сверточную нейросеть использовать как модуль - так же, как используется отдельное простое преобразование `nn.Linear()` или чуть более сложное `nn.Conv2d()`. Удобно воспринимать этот получившийся модуль `resnet152` как некий новый модуль ("слой"), достаточной выразительной способности, чтобы извлекать из входных данных богатые семантически значимые признаки.

При желании можно из таких модулей собрать новую нейросеть. Правда, она будет довольно тяжелой, и смысловая нагрузка такой сети сомнительна.

In [12]:
# Запрещаем вычислять градиенты функции потерь по всем параметрам Resnet152
for param in resnet152.parameters():
    param.requires_grad = False

# Заменяем полносвязную часть на идентичное преобразование,
# чтобы дальше использовать эту сеть как самостоятельный модуль
resnet152.fc = torch.nn.Identity()

_ = resnet152.to(DEVICE)

In [13]:
# Проверим, где располагается модель. Как минимум матрицы параметров.
for param in resnet152.parameters():
    print(param.device)
    break

cuda:0


Теперь на основе полученного модуля `resnet152` создадим новую нейросеть, в которой он будет всего лишь одним из преобразований.

In [14]:
model = torch.nn.Sequential(resnet152,
                            nn.Linear(in_features=2048, out_features=256),
                            nn.ReLU(),
                            nn.Dropout(),
                            nn.Linear(in_features=256, out_features=1))

### дальше - надо обучать модель на нашу новую задачу.